In [3]:
import numpy as np
import matplotlib.pyplot as plt
from bigfile import BigFile
from natsort import natsorted
import glob



## 11/30/2024

In [4]:
snaplist = [760,797,808]
fileroot="/home1/08942/nianyic/scratch3/Astrid/bhdetails/"
for snap in snaplist:
    ifilelist = natsorted(glob.glob(f"{fileroot}/BlackholeDetails-R{snap}*"))
    print(snap)
    for i, file in enumerate(ifilelist):
        bf = BigFile(file)
        redshift = bf["z"][:]
        zmin, zmax = redshift.min(), redshift.max()
        if np.isnan(zmin):
            print(file)
        elif np.isnan(zmax):
            print(file)
        
    


760
797
808


In [2]:
snaplist = [658, 660, 665, 670, 674, 678, 683, 688, 694, 695, 697, 698, 703, 706, 710, 714, 716, 720, 723, 725, 728, 730]
fileroot="/home1/08942/nianyic/scratch3/Astrid/bhdetails/"
for snap in snaplist:
    ifilelist = natsorted(glob.glob(f"{fileroot}/BlackholeDetails-R{snap}*"))
    print(snap)
    for i, file in enumerate(ifilelist):
        bf = BigFile(file)
        redshift = bf["z"][:]
        zmin, zmax = redshift.min(), redshift.max()
        if np.isnan(zmin):
            print(file)
        elif np.isnan(zmax):
            print(file)
        
    


658
660
665
670
674



KeyboardInterrupt



In [3]:
SEL_COL = ['BHID','BHMass','Mdot','Density','timebin','Encounter','MinPos',\
             'MinPot','Entropy','GasVel','acMom','acMass','acBHMass',\
             'Fdbk','SPHID','SwallowID','CountProgs','Swallowed',\
             'BHpos','srDensity','srParticles','srVel','srDisp',\
             'DFAccel','DragAccel','GravAccel','BHvel','Mtrack','Mdyn',\
             'KineticFdbkEnergy','NumDM','V1sumDM','V2sumDM','MgasEnc','KEflag','z']


class BHType:
    def __init__(self, name_sel=None):
        self.name_all = ('size','BHID','BHMass','Mdot','Density','timebin','Encounter','MinPos',\
             'MinPot','Entropy','GasVel','acMom','acMass','acBHMass',\
             'Fdbk','SPHID','SwallowID','CountProgs','Swallowed',\
             'BHpos','srDensity','srParticles','srVel','srDisp',\
             'DFAccel','DragAccel','GravAccel','BHvel','Mtrack','Mdyn',\
             'KineticFdbkEnergy','NumDM','V1sumDM','V2sumDM','MgasEnc','KEflag','z','size2')
        self.dtype_all = ('i','q','d','d','d','i','i','3d',\
            'd','d','3d','3d','d','d',\
            'd','q','q','i','i',\
            '3d','d','d','3d','d',\
            '3d','3d','3d','3d','d','d',\
            'd','d','3d','d','d','i','d','i')

        if name_sel is None:
            name_sel = self.name_all
        self.name_sel = name_sel
        self.name2type = {name: dtype for name, dtype in zip(self.name_all, self.dtype_all)}


    @property
    def TypeAll(self):
        np_type = np.dtype({'names':self.name_all, 'formats':self.dtype_all})
        return np_type
    @property
    def TypeSel(self):
        name_sel = [name for name in self.name_all if name in name_sel]
        dtype_sel = [dtype for name, dtype in zip(self.name_all, self.dtype_all) if name in name_sel]
        np_type = np.dtype({'names':name_sel, 'formats':dtype_sel})
        return np_type

In [4]:

def read_binary_file(file):
    dt = BHTYPE.TypeAll
    dtype = BHTYPE.dtype_all
    #-----------------------------------------------------------
    struct_fmt = ''.join(list(dtype))
    struct_len = struct.calcsize(struct_fmt)
    results = np.fromfile(file, dtype=dt, count=-1)

    results['z'] = 1./results['z'] - 1
    print("rank %d: read %d BHs"%(rank, len(results)))
    assert len(results) == Length[i], "number of BHs read in is not consistent with pre-determined size!"
    print(f"Total memory occupied: {results.nbytes/1e9} GBs")
    return results

In [5]:
def io_binary_file(ifile, ofile):
    dt = BHTYPE.TypeAll
    dtype = BHTYPE.dtype_all
    #-----------------------------------------------------------
    data = np.zeros(1, dtype=BHTYPE.TypeAll)
    nsingle = data.nbytes
    results = np.fromfile(ifile, dtype=dt, count=-1)
    hastrouble = 0

    mask =  (results["size"] != nsingle - 8) | (results["size2"] != nsingle - 8)
    mask = mask.nonzero()[0]
    if len(mask > 0):
        hastrouble = 1
        istart = mask[0]
        ntrouble = sum(mask)
        print(f"starting of trouble {istart}, total troubles {ntrouble}")
        offset = istart * nsingle
        # search for restart point
        for newoff in range(offset + 4, offset + nsingle):
            size1 = np.fromfile(ifile, dtype='i', count=1, offset=newoff)
            size2 = np.fromfile(ifile, dtype='i', count=1, offset=newoff+nsingle-4)
            if (size1 == nsingle - 8) & (size2 == nsingle - 8):
                print(f"resetting offset from {offset} to {newoff}")
                break
        next_data = np.fromfile(ifile, dtype=dt, count=1, offset=newoff)
        assert next_data["size"] == nsingle - 8, f"mismatch of size1 in file {ifile}, {next_data}"
        assert next_data["size2"] == nsingle - 8, f"mismatch of size2 in file {ifile}, {next_data}"
        results = np.fromfile(ifile, dtype=dt, count=-1, offset=newoff)

In [13]:
BHTYPE = BHType(SEL_COL)
data = np.zeros(1, dtype=BHTYPE.TypeAll)
nsingle = data.nbytes

binary_flist = sorted(glob.glob("/home1/08942/nianyic/ASTRID3/BlackholeDetails/BlackholeDetails-R695.2/*"))
print(len(binary_flist))
dt = BHTYPE.TypeAll
dtype = BHTYPE.dtype_all


i = 0

bfile = binary_flist[i]
results = np.fromfile(bfile, dtype=dt, count=-1)
sizelist = results["size"]
mask = sizelist != 428


prob = results[mask]
mask = mask.nonzero()[0]

print(mask[0])

ascale = results[mask[0] - 2]["z"]

print(ascale)


istart = mask[0]
ntrouble = len(mask)
print(f"starting of trouble {istart}, total troubles {ntrouble}")
offset = istart * nsingle

# search for restart point
for newoff in range(offset + 4, offset + nsingle):
    size1 = np.fromfile(bfile, dtype='i', count=1, offset=newoff)
    size2 = np.fromfile(bfile, dtype='i', count=1, offset=newoff+nsingle-4)
    if (size1 == nsingle - 8) & (size2 == nsingle - 8):
        print(f"resetting offset from {offset} to {newoff}")
        break

4096
15858
0.7198807677454985
starting of trouble 15858, total troubles 27613
resetting offset from 6914088 to 6914484


[0.3310427  0.3310427  0.3310427  ... 0.33018296 0.33018296 0.33018296]


In [ ]:
snap = 599
file = "/hildafs/datasets/Asterix/BH_details_bigfile2/BH-Details-R%03d"%snap
bf = BigFile(file)
N = 100000
ids = bf["BHID"][:]
reshift = bf["z"][:]


In [ ]:
idsset = np.unique(ids)
print("Number of unique ids: ", len(idsset))

zset = np.unique(reshift)

In [ ]:
plt.hist(redshift, bins = 20)
plt.show()

In [2]:
snap = 577
file = "/hildafs/datasets/Asterix/BH_details_bigfile2/BH-Details-R%03d"%snap
bf = BigFile(file)
N = 100000
gasvel = bf["GasVel"][:]
# reshift = bf["z"][:]
